In [338]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [339]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [360]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
#baseurl = 'http://127.0.0.1:5000/'

In [361]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [362]:
response.json()

{'signingKey_hex': '052b8889bf51e3932dd5763f1c9a983b76301710f5660f4888692bafa2988e08',
 'traderId': '3',
 'verifyKey_hex': '06bd4131cc7b85979fdcdc2248c5d56e149eda946b259e01b08ba1f323c9b52b'}

In [363]:
url = baseurl + 'createMarket'

mkRtId = 6
mkId=1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [364]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkRtId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [365]:
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,False,1,1,0,6,3


In [366]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk,
                     "traderId": tId,
                     "verifyKey_hex": vk,
                     "marketId": mkId,
                     "price": 0.55,
                     "quantity":1}

In [367]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [368]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': 1,
 'traderId': '3'}

In [369]:
# View json response
pd.DataFrame(response.json(), index=[0])


,checks,marketId,price,quantity,traderId
0,True,1,0.55,1,3


In [370]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [371]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': -1,
 'traderId': '3'}

In [372]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 11.43744683265686 seconds.
matched trade: 1 in 16.77050280570984 seconds.
matched trade: 2 in 11.22226619720459 seconds.
matched trade: 3 in 10.271610021591187 seconds.
matched trade: 4 in 9.977528095245361 seconds.
matched trade: 5 in 10.251175880432129 seconds.
matched trade: 6 in 10.723920106887817 seconds.
matched trade: 7 in 10.101910829544067 seconds.
matched trade: 8 in 9.981605052947998 seconds.
matched trade: 9 in 10.249943017959595 seconds.


In [373]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

failed to create market 1
failed to create market 2
failed to create market 3
failed to create market 4


In [374]:
response.json()

{'checks': 'False',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 3}

In [375]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [376]:
a = pd.read_json(response.json())
a

,marketId,marketRootId,marketBranchId,marketMin,marketMax
0,1,6,1,0,1.0
1,2,1,1,0,0.7
2,3,2,1,0,0.7
3,4,3,1,0,0.7
4,5,4,1,0,0.7


In [377]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [378]:
a = pd.read_json(response.json())
a

,marketId,price,quantity,traderId,iMatched
0,1,0.55,1,1,True
1,1,0.55,-1,1,True
10,1,0.55,-1,1,True
11,1,0.55,1,1,True
12,1,0.55,-1,1,True
13,1,0.55,1,1,True
14,1,0.55,-1,1,True
15,1,0.55,1,1,True
16,1,0.55,-1,1,True
17,1,0.55,1,1,True


In [379]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 5.37952995300293 seconds.
unmatched trade: 1 in 5.298825025558472 seconds.
unmatched trade: 2 in 5.026922941207886 seconds.
unmatched trade: 3 in 4.59512186050415 seconds.
unmatched trade: 4 in 5.402062654495239 seconds.
unmatched trade: 5 in 4.778095006942749 seconds.
unmatched trade: 6 in 5.082495927810669 seconds.
unmatched trade: 7 in 4.674655914306641 seconds.
unmatched trade: 8 in 5.007356882095337 seconds.
unmatched trade: 9 in 10.885777950286865 seconds.


In [380]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [381]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,0.44,2,1
1,1,0.44,10,2
2,1,0.44,10,3


In [382]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [383]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,0.55,0,1
1,1,0.55,0,2
2,1,0.55,0,3
